In [168]:
from models import gmm, lstm
from extractor import extractor
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim

In [169]:
data_wav = "/Users/oosv/data/voxforge"
languages = ["english", "spanish"]
TOTAL_FRAMES = 150
ENERGY_MIN = 12
DELTA_DIM = 3
CEPSTRAL_COEF = 13

extractor.total_frames = TOTAL_FRAMES
extractor.energy_threshold = ENERGY_MIN
files = []
data_y = []
num_samples = 100
#get the path for num_samples .wav files
for i,l in enumerate(languages):
    path = os.path.join(data_wav, l)
    for f in os.listdir(path)[:num_samples]:
        files.append(os.path.join(path, f))
    data_y += [i] * len(os.listdir(path)[:num_samples])

print(len(data_y))
data_x , rejected = np.array(extractor.make_feature_set(files))
print(len(data_x), np.shape(data_x[0]))

200


/Users/oosv/miniconda3/lib/python3.6/site-packages/mkl_fft/_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


200 (150, 3, 13)


In [170]:
input_dim = 39
hidden = 512
languages = 2
frames = 75
batch_size = 10

In [171]:
shape_x, shape_y = np.shape(data_x), np.shape(data_y)
data_x = np.reshape(data_x, [shape_x[0], shape_x[1], shape_x[2] * shape_x[3]])

#data_y = np.reshape(data_y, [shape_y[0], shape_y[1], shape_y[2] * shape_y[3]])

In [172]:
def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [173]:
# Training models on equal amounts of data
shuffle_in_unison(data_x, data_y)
train_x = data_x[int(shape_x[0]/4):]
test_x = data_x[:int(shape_x[0]/4)]
train_y = data_y[int(shape_x[0]/4):]
test_y = data_y[:int(shape_x[0]/4)]

In [174]:
print(np.shape(train_x), np.shape(train_y))
print(np.shape(test_x), np.shape(test_y))

(150, 150, 39) (150,)
(50, 150, 39) (50,)


In [175]:
with open('train_x_sm.npy', 'wb') as file:
    np.save(file, train_x)
with open('train_y_sm.npy', 'wb') as file:
    np.save(file, train_y)
with open('test_x_sm.npy', 'wb') as file:
    np.save(file, test_x)
with open('test_y_sm.npy', 'wb') as file:
    np.save(file, test_y)

In [161]:
importlib.reload(lstm)
_lstm = lstm.LSTM()
# _train_data = torch.tensor(train_data)
# _train_labels = torch.tensor(train_labels)
# print(_train_labels.size())

39
512


In [162]:
def _get_train_data_loader(data, batch_size=100, is_distributed=False, **kwargs):
    
    train_data_x = torch.tensor(data[0], dtype=torch.float32)
    train_data_y = torch.tensor(data[1], dtype=torch.int64)

    if is_distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_data)
    else:
        train_sampler = None

    train_data_x = torch.utils.data.DataLoader(train_data_x, 
                                       batch_size=batch_size, 
                                       shuffle=False,
                                       sampler=train_sampler,
                                       **kwargs)

    train_data_y = torch.utils.data.DataLoader(train_data_y, 
                                       batch_size=batch_size, 
                                       shuffle=False,
                                       sampler=train_sampler,
                                       **kwargs)

    return train_data_x, train_data_y

def _get_test_data_loader(data, batch_size=100, is_distributed=False, **kwargs):
    
    test_data_x = torch.tensor(data[0], dtype=torch.float32)
    test_data_y = torch.tensor(data[1], dtype=torch.int64)
    
    if is_distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_data)
    else:
        train_sampler = None

    test_data_x = torch.utils.data.DataLoader(test_data_x, 
                                       batch_size=batch_size, 
                                       shuffle=False,
                                       sampler=train_sampler,
                                       **kwargs)

    test_data_y = torch.utils.data.DataLoader(test_data_y, 
                                       batch_size=batch_size, 
                                       shuffle=False,
                                       sampler=train_sampler,
                                       **kwargs)
         
    return test_data_x, test_data_y

In [163]:
def train(model_params, train_x, train_y, test_x, test_y):
    # set the seed for generating random numbers
    device = 'cpu'
    train_x, train_y = _get_train_data_loader((train_x, train_y))
    test_x, test_y = _get_test_data_loader((test_x, test_y))    
    model = create_model(model_params, device)
    loss_function = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(),
                        lr=model_params['lr'])
    log_interval = 100
    is_distributed = None
    for epoch in range(2):
        model.train()
        
        for batch_idx, (feature_seq, language) in enumerate(zip(train_x, train_y),1):
            # indicate to use this data with GPU
            print(len(feature_seq))
            print(feature_seq[0].size())
            feature_seq = feature_seq.to(device)
            language = language.to(device)
            
            # zero_grad prevents training a new batch
            # on the last batch's gradient
            optimizer.zero_grad()

            # this calls the forward function, through PyTorch
            # output in shape batch_size x 1 x n_languages
            scores = model(feature_seq)

            # calculate backward loss, get perform gradient descent
            loss = loss_function(scores, language.view(-1))
            loss.backward()
            if is_distributed and not use_cuda:
                # average gradients manually for multi-machine cpu case only
                _average_gradients(model)
            optimizer.step()

            # update logging information
            if batch_idx % log_interval == 0:
                logger.info('Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.sampler),
                    100. * batch_idx / len(train_loader), loss.item()))

In [164]:

def create_model(model_params, device='cpu'):
    '''
    :param languages: languages to train lstm over
    :param model_params: parameters to use for training LSTM
        defaults:
            n_features = 39,
            n_hidden = 512,
            languages = 2, 
            snippet_length = 75,
            dropout=0.0,
            bi_directional=False,
            num_layers = 1,
            linear_layers = 1)
    :return: a LSTM model object containing a lstm per language
    '''
    
    # apparently hyperparms loves to give strings, thus int conversions
    n_features      =   int(model_params.get('n_features', 39))
    n_hidden        =   int(model_params.get('n_hidden', 512))
    languages       =   int(model_params.get('languages', 2)) 
    frames          =   int(model_params.get('frames', 75))
    dropout         =   int(model_params.get('dropout', 0.0))
    num_lstm_layers =   int(model_params.get('num_layers', 1))
    bidirectional   =   bool(model_params.get('bidirectional', False))
    num_linear      =   int(model_params.get('num_linear', 1))

    # if there is a gpu, the LSTM will take care of checking for that during training
    return lstm.LSTM(
        n_features,
        n_hidden,
        languages,
        frames,
        dropout,
        bidirectional,
        num_lstm_layers
    ).to(device)

In [165]:
model_params={
    'n_features' : 39,
    'n_hidden'   : 50, 
    'languages'  : 2,
    'frames'     : TOTAL_FRAMES,
    'dropout'    : 0,
    'num_layers' : 1, 
    'bidirectional' : False,
    'momentum'   : 0.01,
    'lr'         : 0.001
}

In [166]:
importlib.reload(lstm)
train(model_params, train_x, train_y, test_x, test_y)

39
50
100
torch.Size([150, 39])
50
torch.Size([150, 39])
100
torch.Size([150, 39])
50
torch.Size([150, 39])


In [1]:

# Define IAM role
import boto3
import re
import os
import numpy as np
import pandas as pd
import importlib
from sagemaker import get_execution_role
import sagemaker as sage
from time import gmtime, strftime
from sagemaker.pytorch import PyTorch
import time

role         = get_execution_role()
sess         = sage.Session()
bucket       = 'oosv-multilingual-bucket'
TOTAL_FRAMES = 150

In [2]:
def get_data(file_name, bucket, _dir='train'):
    prefix = '/tmp/data/'
    path = f'data/{_dir}/'
    s3 = boto3.resource('s3')
    s3.Bucket(bucket).download_file(path + file_name, prefix + file_name)
    arr = np.load(prefix + file_name)
    os.remove(prefix + file_name)
    return arr

In [10]:
def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [29]:
train_x = get_data('train_x.npy', bucket)
train_y = get_data('train_y.npy', bucket)
test_x  = get_data('test_x.npy', bucket, 'test')
test_y  = get_data('test_y.npy', bucket, 'test')
print(np.shape(train_x))
print(np.shape(train_y))
print(np.shape(test_x))
print(np.shape(test_y))

(68223, 150, 39)
(68223,)
(22741, 150, 39)
(22741,)


In [31]:
from collections import Counter
c = Counter(train_y)
print(c)
print(train_y[45115])
train_x_tmp = train_x[:c[1]]
train_x_tmp = np.concatenate((train_x_tmp, train_x[c[0]:]), axis=0)
print(np.shape(train_x_tmp))
train_y_tmp = train_y[:c[1]]
train_y_tmp = np.concatenate((train_y_tmp , train_y[c[0]:]), axis=0)
print(len(train_y_tmp))

Counter({0: 45116, 1: 23107})
0
(46214, 150, 39)
46214


In [33]:
shuffle_in_unison(train_x_tmp, train_y_tmp)
train_x = train_x_tmp
train_y = train_y_tmp
with open('train_x.npy', 'wb') as file:
    np.save(file, train_x)
with open('train_y.npy', 'wb') as file:
    np.save(file, train_y)


In [36]:
def upload_to_bucket(file):
    path = f'data/train/'
    s3 = boto3.resource('s3')
    s3.Bucket(bucket).upload_file(file, path + file)
upload_to_bucket('train_x.npy')
upload_to_bucket('train_y.npy')